In [ ]:
import os, glob
import numpy as np
import pandas as pd
from datetime import time
from pytz import timezone
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [ ]:
# 1) —— Data Import & Preprocessing —— #

DATA_DIR = "../SPX"
OUTPUT_FEATURE_CSV = "../csvfiles_python/features_360_raw.csv"
OUTPUT_ALLSET_CSV  = "../csvfiles_python/allSet_360_raw.csv"

def load_data(path):
    files = glob.glob(os.path.join(path, "*.txt"))
    dfs = []
    for f in sorted(files):
        df = pd.read_csv(f, names=["DateTime","Open","High","Low","Close"],
                         sep=",", parse_dates=["DateTime"])
        dfs.append(df)
    spx = pd.concat(dfs, ignore_index=True)
    # localize to EST
    spx["DateTime"] = spx["DateTime"].dt.tz_localize(timezone("US/Eastern"))
    spx.set_index("DateTime", inplace=True)
    # filter trading hours 9:30–16:00
    return spx.between_time("09:30","16:00")

spx = load_data(DATA_DIR)

# Drop duplicates
spx = spx[~spx.index.duplicated(keep='first')]

In [ ]:
# 2) —— Create daily labels —— #

# group by calendar date
spx["Date"] = spx.index.date
groups = spx.groupby("Date")

# last-minute close price per day
lmP = groups["Close"].last()

# average price from start to (end − 30 mins)
avgP = groups.apply(lambda df: df["Close"].iloc[:-30].mean())

# binary label: 1 if avgP < lmP else 0
y = (avgP < lmP).astype(int)
y.name = "Y"